# Text Analytics Assignment Two
### October 8
#### Luke Bravo (section B), Pete Davis (sec. A), Benjamin Deutsch (sec. A), Khoi Tran (sec. A)

In [1]:
import pandas as pd
import numpy as np
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from collections import Counter

## TASK A
We used selenium scraper code to extract beer names, beer ratings, and beer reviews for 250 beers on https://www.beeradvocate.com/beer/top-rated/, and we converted this to a .csv file. Below, we have converted the .csv file to a dataframe, and we will be using this data for our analysis. 

In [2]:
df = pd.read_csv("beers.csv")
df.drop('Unnamed: 0', axis = 1, inplace = True)
df = df.sort_values(by = 'user_rating', ascending = False)
df

,product_name,product_review,user_rating
0,Kentucky Brunch Brand Stout,Smell: early morning pancakes and coffee befo...,5.00
368,Pliny The Elder,Just enjoyed a very fresh (10-day-old) bottli...,5.00
387,Juice Machine,Look: A red-hued Julius. Vibrant and bright. ...,5.00
2608,Miami Madness,5's across the board. No doubt about it. This...,5.00
2635,Beyond Good And Evil,Fantastic beer. Dark brown to black appearanc...,5.00
...,...,...,...
2898,Pirate Paradise,I hate this beer. I just paid $10 for 1 12 ou...,2.31
5168,Rodenbach Caractère Rouge,"750ml corked bottle from the LCBO, served fro...",2.03
442,Assassin,"Wow, very disappointed in this one. I would n...",1.86
2,Kentucky Brunch Brand Stout,It's hyped... There is a lot of breweries doi...,1.49


## TASK B

We are assuming the customer has specified three attributes: 
- crisp 
- complex
- hoppy

In [3]:
stop = stopwords.words('english')
words = df['product_review'].map(lambda x: word_tokenize(str(x).lower()))
clean = words.apply(lambda x: [item for item in x if item not in stop])
clean = clean.apply(lambda x: [item for item in x if len(item) > 2])
clean = clean.apply(lambda x: [item for item in x if ":" not in item])
clean = clean.apply(lambda x: [item for item in x if "," not in item])
clean = clean.apply(lambda x: [item for item in x if "." not in item])
clean = clean.apply(lambda x: [item for item in x if "..." not in item])

#creates a list of lists that only include a word once for a given message (reduces artificial inflation)
filtered = []
for i in range(len(clean)):
    filtered.append(list(dict.fromkeys(clean[i])))
this = pd.Series(filtered)
this

0       [smell, early, morning, pancakes, coffee, work...
1       [2019, vintage, pours, dark, brown, color, not...
2       [hyped, lot, breweries, style, beer, right, to...
3       [reviewing, 2019, vintage, pours, thick, close...
4       [2018, version, poured, dark, small, head, sme...
                              ...                        
6225    l, bottle, thanks, dustin, poured, taster...
6226    [reviewed, notes, 7/27/14, pours, deep, glowin...
6227    [500, bottle, poured, snifter, appearance, nic...
6228    [poured, 500, bottle, snifter, reviewed, notes...
6229    [lovely, dark, blueberry, colour, hints, ruby,...
Length: 6230, dtype: object

In [4]:
#get binary representation of word presence
count = this.map(lambda x: Counter(x))
messages = count
messages 

#get 
final = count[0]
for i in range(1,len(count)):
    final += count[i]
frequencydf = pd.DataFrame.from_dict(final,orient='index').reset_index()
frequencydf = frequencydf.rename(columns = {'index':'Word', 0:'Count'})
frequencydf = frequencydf.sort_values(by = 'Count', ascending = False)
frequencydf[:10]

,Word,Count
97,head,3513
55,beer,2820
7,taste,2698
38,nose,1723
29,pours,1704
248,sweet,1697
15,like,1655
154,one,1645
30,dark,1640
236,finish,1624


## Task C

In [5]:
def cos_sim_finder (review,attribute):
    X = review
    Y = attribute

    # tokenization 
    X_list = word_tokenize(X)  
    Y_list = word_tokenize(Y) 

    # sw contains the list of stopwords 
    sw = stopwords.words('english')  
    l1 =[];l2 =[] 

    # remove stop words from the string 
    X_set = {w for w in X_list if not w in sw}  
    Y_set = {w for w in Y_list if not w in sw} 

    # form a set containing keywords of both strings  
    rvector = X_set.union(Y_set)  
    for w in rvector: 
        if w in X_set: l1.append(1) # create a vector 
        else: l1.append(0) 
        if w in Y_set: l2.append(1) 
        else: l2.append(0) 
    c = 0

    # cosine formula  
    for i in range(len(rvector)): 
            c+= l1[i]*l2[i] 
    if sum(l1)*sum(l2)!=0:
        cosine = c / float((sum(l1)*sum(l2))**0.5) 
        #print("similarity: ", cosine) 
        #crisp_sims.append(cosine)
        return cosine
    else:
        pass

### Cosine Similarity Scores

In [110]:
#add temporary columns to df with each attribute's similarity
similarity_crisp=[]
similarity_complex=[]
similarity_hoppy=[]

##### WE DIDNT MAKE THE INPUT A FILE
for i in range(len(df)):
    similarity_crisp.append(cos_sim_finder(df['product_review'].iloc[i],"crisp"))
    similarity_complex.append(cos_sim_finder(df['product_review'].iloc[i],"complex"))
    similarity_hoppy.append(cos_sim_finder(df['product_review'].iloc[i],"hoppy"))

df['similarity_crisp']=similarity_crisp
df['similarity_complex']=similarity_complex
df['similarity_hoppy']=similarity_hoppy


#### we are supposed to drop user rating here, but we need it later!!!
#calculate average similarity and drop temp columns
df['avg_similarity']=((df['similarity_crisp']+df['similarity_complex']+df['similarity_hoppy'])/3)
df.drop(['similarity_crisp','similarity_complex','similarity_hoppy'],axis=1,inplace=True)
df_sim = df.copy(deep=True)
df_sim.drop(['user_rating'], axis = 1, inplace = True)

df_sim.sort_index(axis=0,inplace=True)
df_sim.to_csv('Part C Output1.csv',index=False)
df_sim.sort_values(by = 'avg_similarity',ascending = False)[:10]

,product_name,product_review,avg_similarity
3267,Pseudo Sue,"A very crisp, very hoppy pale ale. Definitely...",0.142134
3351,Permanent Funeral,"I love a creamy, citrusy IPA and to me this i...",0.142134
4087,Oude Geuze Golden Blend,"2014 vintage, poured from a 375. Pours a gold...",0.142134
2371,Trappistes Rochefort 10,The only beer in the world I gave 5 on taste....,0.139010
4431,Bad Boy,Crystal clear yellow with a dark white head. ...,0.109599
1023,Doubleganger,Quick review - on tap at Tree House: Look (4....,0.104116
5688,Peche 'n Brett,Poured from a 500ml bottle into a tulip glass...,0.100504
3162,Hommage,Bright reddish-copper with a thick fuzzy whit...,0.098295
1473,Nectarine Premiere,Bottled 6/19 Poured from Cellar temp on 3/20 ...,0.096225
1166,Headroom,"My first Trillium beer, have to figure out ho...",0.095238


## Task D

In [111]:
#create lists of rows containing each attribute
rows_crisp = []
for i in range(len(this)):
    for j in range(len(this[i])):
        if 'crisp' in this[i][j]: #this will include variants (e.g., crisp-ier, crisp-y)
            rows_crisp.append(i)
        else:
            pass
rows_crisp = list(dict.fromkeys(rows_crisp)) #removes duplicates

rows_hoppy = []
for i in range(len(this)):
    for j in range(len(this[i])):
        if 'hopp' in this[i][j]: #this will include variants (e.g., hopp-iness, hopp-y)
            rows_hoppy.append(i)
        else:
            pass        
rows_hoppy = list(dict.fromkeys(rows_hoppy)) #removes duplicates

rows_complex = []
for i in range(len(this)):
    for j in range(len(this[i])):
        if 'complex' in this[i][j]: #this will include variants (e.g., complex-ity)
            rows_complex.append(i)
        else:
            pass  
rows_complex = list(dict.fromkeys(rows_complex)) #removes duplicates

In [112]:
crisp_mask = []
for i in range(len(this)):
    if i in rows_crisp:
        crisp_mask.append(True)
    else:
        crisp_mask.append(False)
crisp_rows = this[crisp_mask]

hoppy_mask = []
for i in range(len(this)):
    if i in rows_hoppy:
        hoppy_mask.append(True)
    else:
        hoppy_mask.append(False)
hoppy_rows = this[hoppy_mask]

complex_mask = []
for i in range(len(this)):
    if i in rows_complex:
        complex_mask.append(True)
    else:
        complex_mask.append(False)
complex_rows = this[complex_mask]

#crisp_rows has all the rows with crisp in it
#hoppy_rows has all the rows with hoppy in it
#complex_rows has all the rows with complex in it
#these are referenced later as keys for merging sentiment scores back into our original df

In [115]:
#rename columns for reference in the following functions
complex_rows2=complex_rows.reset_index().drop('index',axis=1).rename(columns = {0:'reviews'})
complex_reviews=complex_rows2['reviews']

crisp_rows2=crisp_rows.reset_index().drop('index',axis=1).rename(columns = {0:'reviews'})
crisp_reviews=crisp_rows2['reviews']

hoppy_rows2=hoppy_rows.reset_index().drop('index',axis=1).rename(columns = {0:'reviews'})
hoppy_reviews=hoppy_rows2['reviews']

In [116]:
def get_complex_windows (review):
    complex_windows=[]
    for j in range(len(review)):
        if 'complex' in review[j]:
            if j < 3:
                w=0
            else:
                w=j-3
            complex_windows.append(review[w:j+4])
    return complex_windows[0]

def get_crisp_windows (review):
    crisp_windows=[]
    for j in range(len(review)):
        if 'crisp' in review[j]:
            if j < 3:
                w=0
            else:
                w=j-3
            crisp_windows.append(review[w:j+4])
    return crisp_windows[0]

def get_hoppy_windows (review):
    hoppy_windows=[]
    for j in range(len(review)):
        if 'hopp' in review[j]:
            if j < 3:
                w=0
            else:
                w=j-3
            hoppy_windows.append(review[w:j+4])
    return hoppy_windows[0]

complex_window = complex_reviews.map(get_complex_windows)
crisp_window = crisp_reviews.map(get_crisp_windows)
hoppy_window = hoppy_reviews.map(get_hoppy_windows)

In [117]:
!pip install vaderSentiment

#import SentimentIntensityAnalyzer class SentimentIntensityAnalyzer
#from vaderSentiment.vaderSentiment module 
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer 
  
# function to print sentiments 
# of the sentence. 
def sentiment_scores(sentence): 
  
    # Create a SentimentIntensityAnalyzer object. 
    sid_obj = SentimentIntensityAnalyzer() 
  
    # polarity_scores method of SentimentIntensityAnalyzer 
    # oject gives a sentiment dictionary. 
    # which contains pos, neg, neu, and compound scores. 
    sentence = ' '.join(sentence)
    sentiment_dict = sid_obj.polarity_scores(sentence) 

    return(sentiment_dict['compound'])

In [118]:
#get sentiment scores of out attribute window phrases
scores_complex=complex_window.map(sentiment_scores)
scores_crisp=crisp_window.map(sentiment_scores)
scores_hoppy=hoppy_window.map(sentiment_scores)

In [119]:
#make temporary dataframes sentiment scores for each attribute paired with corresponding rows from OG df
complextempdf=pd.DataFrame(data=rows_complex)
complextempdf['sent_complex']=scores_complex
#complextempdf['attribute_tag']='complex'
complextempdf.rename(columns = {0:'OG_row'},inplace=True)

crisptempdf=pd.DataFrame(data=rows_crisp)
crisptempdf['sent_crisp']=scores_crisp
#crisptempdf['attribute_tag']='crisp'
crisptempdf.rename(columns = {0:'OG_row'},inplace=True)

hoppytempdf=pd.DataFrame(data=rows_hoppy)
hoppytempdf['sent_hoppy']=scores_hoppy
#hoppytempdf['attribute_tag']='hoppy'
hoppytempdf.rename(columns = {0:'OG_row'},inplace=True)

tempframes=[complextempdf,crisptempdf,hoppytempdf]
tempframes=pd.concat(tempframes)

In [120]:
#merging the attribute score df's with the original df
premergeddf=df.reset_index()
premergeddf.rename(columns = {'index':'OG_row'},inplace=True)

### Average Sentiment for Each Review

In [121]:
tempframes=tempframes.groupby('OG_row').sum()
tempframes.reset_index(inplace=True)

mergeddf=premergeddf.merge(tempframes,how='outer',on='OG_row')

# I am making 0's NaN
mergeddf=mergeddf.replace(0,np.nan)
mean_df = mergeddf[['sent_complex', 'sent_crisp', 'sent_hoppy']]

averages = mean_df.mean(axis = 1, skipna = True)
averages
mergeddf['avg_sent'] = averages

mergeddf.sort_values(by = 'avg_sent', ascending = False)

,OG_row,product_name,product_review,user_rating,avg_similarity,sent_complex,sent_crisp,sent_hoppy,avg_sent
2691,2319.0,Breakfast Stout,Hey for you stout lovers what’s not to like. ...,4.55,0.062994,0.9274,NaN,NaN,0.9274
3223,5165.0,Rodenbach Caractère Rouge,2013 vintage. Poured from a bomber into a Dog...,4.50,0.037503,0.9217,NaN,NaN,0.9217
134,857.0,Very GGGreennn,That is possibly my favorite tree house beer ...,5.00,0.065372,NaN,NaN,0.91,0.9100
2790,2169.0,Ghost In The Machine - Double Dry-Hopped,Picture perfect hazy yellow body and pearl wh...,4.54,0.046676,NaN,0.9042,NaN,0.9042
3607,5383.0,Expedition Stout - Bourbon Barrel-Aged,Review #1200 Amazing Russian imperial stout ...,4.47,0.037268,0.9001,NaN,NaN,0.9001
...,...,...,...,...,...,...,...,...,...
6225,2898.0,Pirate Paradise,I hate this beer. I just paid $10 for 1 12 ou...,2.31,NaN,NaN,NaN,NaN,NaN
6226,5168.0,Rodenbach Caractère Rouge,"750ml corked bottle from the LCBO, served fro...",2.03,NaN,NaN,NaN,NaN,NaN
6227,442.0,Assassin,"Wow, very disappointed in this one. I would n...",1.86,NaN,NaN,NaN,NaN,NaN
6228,2.0,Kentucky Brunch Brand Stout,It's hyped... There is a lot of breweries doi...,1.49,NaN,NaN,NaN,NaN,NaN


### Average Sentiment For Each Product

In [122]:
mergeddf=mergeddf.replace(np.nan,0) 
productlevel=mergeddf.drop('OG_row', axis=1).groupby('product_name').mean()
productlevel.to_csv('Task D Output.csv')
productlevel.sort_values(by = 'avg_sent', ascending = False)

,user_rating,avg_similarity,sent_complex,sent_crisp,sent_hoppy,avg_sent
product_name,,,,,,
Dorothy (Wine Barrel Aged),4.6120,0.021151,0.230604,0.041916,0.018352,0.290872
Rodenbach Caractère Rouge,4.3304,0.015400,0.199288,0.010928,0.000000,0.192764
Zenne Y Frontera,4.7068,0.010505,0.175276,0.000000,0.000000,0.175276
Ghost In The Machine - Double Dry-Hopped,4.5044,0.012154,0.017616,0.040276,0.106332,0.164224
Flora,4.4852,0.011034,0.123504,0.033692,0.000000,0.157196
...,...,...,...,...,...,...
Fuzzy Baby Ducks,4.3100,0.006605,0.000000,0.000000,-0.016076,-0.016076
Wide Awake It's Morning,4.3852,0.004878,0.004872,-0.021692,0.000000,-0.016820
Citraquench'l,4.4724,0.004970,0.000000,0.000000,-0.016860,-0.016860


## Task E

In [133]:
productleveleval=productlevel
productleveleval=productleveleval.replace(np.nan,0)
productleveleval['eval_score']=productleveleval['avg_similarity']+productleveleval['avg_sent']
productleveleval=productleveleval.sort_values(by='eval_score',ascending=False).iloc[:3]
productleveleval.reset_index(inplace=True)
productleveleval[:3]

,product_name,user_rating,avg_similarity,sent_complex,sent_crisp,sent_hoppy,avg_sent,eval_score
0,Dorothy (Wine Barrel Aged),4.6120,0.021151,0.230604,0.041916,0.018352,0.290872,0.312023
1,Rodenbach Caractère Rouge,4.3304,0.015400,0.199288,0.010928,0.000000,0.192764,0.208164
2,Zenne Y Frontera,4.7068,0.010505,0.175276,0.000000,0.000000,0.175276,0.185781


In [134]:
productleveleval[['product_name','eval_score']]

,product_name,eval_score
0,Dorothy (Wine Barrel Aged),0.312023
1,Rodenbach Caractère Rouge,0.208164
2,Zenne Y Frontera,0.185781


### Top 3 Beers
The top 3 recommended beers based on the evaluation scores are:
    1. Dorothy (Wine Barrel Aged)
    2. Rodenbach Caractère Rouge
    3. Zenne Y Frontera

## Task F

In [135]:
!pip install -U spacy

!python -m spacy download en_core_web_sm

import spacy
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd
import en_core_web_sm
nlp = en_core_web_sm.load()

Requirement already up-to-date: spacy in c:\users\19727\anaconda3\lib\site-packages (2.3.2)
[+] Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')


### spaCy

In [137]:
spacy_cos_sims=[]
attributes='complex crisp hoppy'

for i in range(0,6230):
    cos = [df["product_review"].iloc[i],attributes]
    count_vectorizer = CountVectorizer(stop_words='english')
    sparse_matrix = count_vectorizer.fit_transform(cos)
    doc_term_matrix = sparse_matrix.todense()
    dfs1 = pd.DataFrame(doc_term_matrix, columns = count_vectorizer.get_feature_names(), index =['x','y'])
    spacy_cos_sims.append(cosine_similarity(dfs1,dfs1)[0,1])

In [138]:
#create spacydf

spacydf=mergeddf.copy(deep=True)
spacydf['avg_similarity']=spacy_cos_sims
spacydf=spacydf.rename(columns = {'avg_similarity':'spacy_similarity'})

In [139]:
mean_df = spacydf[['sent_complex', 'sent_crisp', 'sent_hoppy']]
averages = mean_df.mean(axis = 1, skipna = True)
averages
spacydf['avg_sent'] = averages
spacyproductlevel=spacydf.drop('OG_row', axis=1).groupby('product_name').mean()
#productlevel.to_csv('Task D Output.csv')
spacyproductlevel

spacyproductleveleval=spacyproductlevel
spacyproductleveleval=spacyproductleveleval.replace(np.nan,0)
spacyproductleveleval['eval_score']=spacyproductleveleval['spacy_similarity']+spacyproductleveleval['avg_sent']
spacyproductleveleval=spacyproductleveleval.sort_values(by='eval_score',ascending=False).iloc[:3]
spacyproductleveleval.reset_index(inplace=True)

spacyproductleveleval

,product_name,user_rating,spacy_similarity,sent_complex,sent_crisp,sent_hoppy,avg_sent,eval_score
0,Dorothy (Wine Barrel Aged),4.6120,0.043944,0.230604,0.041916,0.018352,0.096957,0.140901
1,Rodenbach Caractère Rouge,4.3304,0.022913,0.199288,0.010928,0.000000,0.070072,0.092985
2,Pliny The Younger,4.7204,0.034446,0.028012,0.056084,0.082648,0.055581,0.090028


In [130]:
spacyproductleveleval[['product_name','spacy_similarity']].merge(productleveleval[['product_name','avg_similarity']], on = 'product_name')

,product_name,spacy_similarity,avg_similarity
0,Dorothy (Wine Barrel Aged),0.043944,0.021151
1,Rodenbach Caractère Rouge,0.022913,0.015400


### Recommendation - SpaCy vs. Bag-of-Words
Top three recommendations using spaCy cosine similarities to compute evaluation score:
    1. Dorothy (Wine Barrel Aged)
    2. Rodenbach Caractère Rouge	
    3. Pliny The Younger

If we used spaCy similarity instead of bag-of-words cosine similarity, our top two beer recommendations would be the same, but our third beer recommendation would be different. Using cosine similarity, the beer with the thrid highest evaluation score was Zenne Y Frontera, and when we used spaCy, it was Pliny The Younger. 

There is certainly a difference in the similarity scores that were calculated using the two approaches, and that is what is changing our recommendation. Above, we have showed the respective similarity scores for the beers that overlapped in our recommendation. As you can see, the two yielded different results. 

In [131]:
#spaCy percentages

print('spaCy percentages')

spacydorothy=spacydf[spacydf['product_name']=='Dorothy (Wine Barrel Aged)']
print(len(spacydorothy[spacydorothy['avg_sent']!=0])/len(spacydorothy)*100,'%')

spacyrod=spacydf[spacydf['product_name']=='Rodenbach Caractère Rouge']
print(len(spacyrod[spacyrod['avg_sent']!=0])/len(spacyrod)*100,'%')

spacypliny=spacydf[spacydf['product_name']=='Pliny The Younger']
print(len(spacypliny[spacypliny['avg_sent']!=0])/len(spacypliny)*100,'%')

#bag-of-words percentages

print('\nBag-of-Words percentages')

ogdorothy=mergeddf[mergeddf['product_name']=='Dorothy (Wine Barrel Aged)']
print(len(ogdorothy[ogdorothy['avg_sent']!=0])/len(ogdorothy)*100,'%')

ogrod=mergeddf[mergeddf['product_name']=='Rodenbach Caractère Rouge']
print(len(ogrod[ogrod['avg_sent']!=0])/len(ogrod)*100,'%')

ogpliny=mergeddf[mergeddf['product_name']=='Pliny The Younger']
print(len(ogpliny[ogpliny['avg_sent']!=0])/len(ogpliny)*100,'%')

spaCy percentages
56.00000000000001 %
40.0 %
36.0 %

Bag-of-Words percentages
56.00000000000001 %
40.0 %
36.0 %


### Percentage of Reviews Mentioning Attributes

In our analysis, we did not find a difference in percentages between the spaCy and bag-of-words approaches. However, this could be because there are only 25 reviews per product, and the chances of finding synonyms are small for this small sample size.

## Task G

In [132]:
productlevel.sort_values(by='user_rating',ascending=False)

,user_rating,avg_similarity,sent_complex,sent_crisp,sent_hoppy,avg_sent
product_name,,,,,,
Chemtrailmix,4.754286,0.000000,0.000000,0.000000,0.000000,0.000000
It Was All A Dream,4.747200,0.000000,0.021068,0.000000,0.000000,0.021068
SR-71,4.746400,0.000000,0.000000,0.000000,0.000000,0.000000
Barrel-Aged Abraxas,4.744400,0.003654,0.024384,0.000000,0.000000,0.024384
Ann,4.725200,0.007234,0.111972,0.000000,0.000000,0.111972
...,...,...,...,...,...,...
La Fosse,4.288400,0.003349,-0.016076,0.000000,0.000000,-0.016076
Hold On To Sunshine,4.252000,0.004581,0.014448,-0.013600,0.000000,0.000848
Mastermind,4.224400,0.009049,0.017616,0.000000,-0.007116,0.005250


### Recommendation - User Rating Only

Our recommendation would completely change if we chose the 3 highest products from our entire data set.
The top 3 would be:
    1. Chemtrailmix       -> User Rating = 4.754286
    2. It Was All A Dream -> User Rating = 4.74720
    3. SR-71              -> User Rating = 4.74640
There was no overlap between the top three beer recommendations using the evaluation score versus using the average user rating. In addition, not all of these beers would meet the requirements of the user looking for recommendations. In fact, It Was All A Dream was the only beer of the three to have reviews relating to any of the attributes, and the only attribute that it related to was complex.